# Identifying arXiv Article Subject Codes via NLP

The goal of this project is to predict primary subject codes for scientific articles available in the arXiv database based on the text of their abstract. This allows for rapid encoding of article subject material, and similar methods may be applicable for identifying key terms in articles submitted for addition to the database. 

Without machine learning, idexing articles for addition to a database along subject codes, key terms, and other metrics is a labor intensive process. For some databases, the labor cost of indexing a single article has been estimated to cost up to 10 dollars per article. Natural Language Processing offers the ability to automate this process, thereby saving up to 15 million dollars in labor for a dataset similar in size to the one used for this project upon intial upload. Maintenance costs may see further cost savings as routine updates to the database architecture or indexing system can be automated instead of requiring large quantities of expensive manual labor.

In [11]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

import nltk

In [2]:
#import data
df = pd.read_csv("./arxiv-oai-af.tsv", delimiter="\t")
df

,abstract,acm_class,arxiv_id,author_text,categories,comments,created,doi,num_authors,num_categories,primary_cat,title,updated
0,If we assume the Thesis that any classical T...,NaN,math/0212388,Bhupinder Singh Anand,math.GM,12 pages. Revision 1. Appendix 1 added. An HTM...,2002-12-31,NaN,1,1,math.GM,Is a deterministic universe logically consiste...,2003-01-02
1,"We define the Cartesian product, composition...",NaN,1205.6123,"Muhammad Akram, Wieslaw A. Dudek",cs.DM,NaN,2012-04-29,10.1016/j.camwa.2010.11.004,2,1,cs.DM,Interval-valued fuzzy graphs,NaN
2,We apply algebraic Morse theory to the Taylo...,NaN,1806.07887,Robin Frankhuizen,"math.AT,math.AC,math.RA",27 pages; comments welcome. arXiv admin note: ...,2018-06-20,NaN,1,3,math.AT,Massey products and the Golod property for sim...,NaN
3,Anomalous transport is usually described eit...,NaN,1007.3022,"Bartlomiej Dybiec, Ewa Gudowska-Nowak","cond-mat.stat-mech,math-ph,math.MP","10 pages, 7 figures",2010-07-18,10.1063/1.3522761,2,3,cond-mat.stat-mech,Subordinated diffusion and CTRW asymptotics,2010-11-09
4,"In this paper, an approximate solution to a ...",NaN,1512.07787,"M. T. Araujo, E. Drigo Filho",cond-mat.stat-mech,"12 pages, 8 figures",2015-12-24,10.5488/CMP.18.43003,2,1,cond-mat.stat-mech,Approximate solution for Fokker-Planck equation,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1582236,21cm intensity mapping experiments aim to ob...,NaN,1501.03823,"Laura Wolz, Filipe B. Abdalla, David Alonso, C...",astro-ph.CO,This article is part of the 'SKA Cosmology Cha...,2015-01-15,NaN,10,1,astro-ph.CO,Foreground Subtraction in Intensity Mapping wi...,NaN
1582237,We show the existence of smooth isolated cur...,NaN,math/0110220,Andreas Leopold Knutsen,math.AG,18 pages. The previous version of the preprint...,2001-10-19,NaN,1,1,math.AG,"Smooth, isolated curves in families of Calabi-...",2012-09-05
1582238,Sequence alignment is a tool in bioinformati...,NaN,0907.2187,"S Wolfsheimer, O Melchert, AK Hartmann","cond-mat.stat-mech,cond-mat.dis-nn,q-bio.QM",NaN,2009-07-13,10.1103/PhysRevE.80.061913,3,3,cond-mat.stat-mech,Finite-temperature local protein sequence alig...,NaN
1582239,"We suggest that the majority of the ""young"",...",NaN,astro-ph/0209553,Valery V. Kravtsov,astro-ph,"7 pages, no figures, accepted for publication ...",2002-09-26,10.1051/0004-6361:20021404,1,1,astro-ph,Second Parameter Globulars and Dwarf Spheroida...,NaN


In [3]:
#find explicit nulls
df.isnull().sum()

abstract                0
acm_class         1560822
arxiv_id                0
author_text             0
categories              0
comments           301829
created                 0
doi                734897
num_authors             0
num_categories          0
primary_cat             0
title                   0
updated            991881
dtype: int64

In [4]:
#drop unnecessary columns 
trimmed_df = df.drop(columns=["acm_class", 
                              "comments", 
                              "created", 
                              "num_authors", 
                              "num_categories", 
                              "updated", 
                              "doi"]
                    )
trimmed_df

,abstract,arxiv_id,author_text,categories,primary_cat,title
0,If we assume the Thesis that any classical T...,math/0212388,Bhupinder Singh Anand,math.GM,math.GM,Is a deterministic universe logically consiste...
1,"We define the Cartesian product, composition...",1205.6123,"Muhammad Akram, Wieslaw A. Dudek",cs.DM,cs.DM,Interval-valued fuzzy graphs
2,We apply algebraic Morse theory to the Taylo...,1806.07887,Robin Frankhuizen,"math.AT,math.AC,math.RA",math.AT,Massey products and the Golod property for sim...
3,Anomalous transport is usually described eit...,1007.3022,"Bartlomiej Dybiec, Ewa Gudowska-Nowak","cond-mat.stat-mech,math-ph,math.MP",cond-mat.stat-mech,Subordinated diffusion and CTRW asymptotics
4,"In this paper, an approximate solution to a ...",1512.07787,"M. T. Araujo, E. Drigo Filho",cond-mat.stat-mech,cond-mat.stat-mech,Approximate solution for Fokker-Planck equation
...,...,...,...,...,...,...
1582236,21cm intensity mapping experiments aim to ob...,1501.03823,"Laura Wolz, Filipe B. Abdalla, David Alonso, C...",astro-ph.CO,astro-ph.CO,Foreground Subtraction in Intensity Mapping wi...
1582237,We show the existence of smooth isolated cur...,math/0110220,Andreas Leopold Knutsen,math.AG,math.AG,"Smooth, isolated curves in families of Calabi-..."
1582238,Sequence alignment is a tool in bioinformati...,0907.2187,"S Wolfsheimer, O Melchert, AK Hartmann","cond-mat.stat-mech,cond-mat.dis-nn,q-bio.QM",cond-mat.stat-mech,Finite-temperature local protein sequence alig...
1582239,"We suggest that the majority of the ""young"",...",astro-ph/0209553,Valery V. Kravtsov,astro-ph,astro-ph,Second Parameter Globulars and Dwarf Spheroida...


In [5]:
#check for duplicate articles
trimmed_df["arxiv_id"].value_counts()

hep-th/9306137      1
1309.2498           1
1704.03351          1
math/0310105        1
1805.08534          1
                   ..
1605.05538          1
1204.4580           1
1010.3138           1
astro-ph/0211316    1
1309.4655           1
Name: arxiv_id, Length: 1582241, dtype: int64

Train-test split

In [6]:
train, test = train_test_split(trimmed_df, test_size=0.2, random_state=42)

In [7]:
#save test data for later
test.to_csv("./Train&Test/test_split")

In [13]:
#save train data for easy import during development time
train.to_csv("./Train&Test/train_split")

### Pipeline

In [ ]:
#Creating a preprocessor function that can be integrated into the pipeline
Abstract_NL_Processor = FunctionTransformer

In [12]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
np.array([sample_text])

array(['The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.'],
      dtype='<U104')

### Model Construction

### Model Iteration

### Model Evaluation

# Discussion